In [1]:
import sqlite3

In [ ]:
# Check the API version.
sqlite3.version_info

In [ ]:
# Check the underlying SQLite version.
sqlite3.sqlite_version

In [ ]:
# Connect to an in-memory database.
connection = sqlite3.connect(":memory:")
connection

In [ ]:
# Create a Cursor object.
cursor = connection.cursor()
cursor

In [ ]:
# Attributes from the Cursor.
# - execute methods -> Perform SQL statements.
# - fetch methods   -> Fetch data from a SELECT resultset.
# - description     -> Metadata for a SELECT resultset.
# - rowcount        -> Rows affected by the last SQL statement.
for attribute in dir(cursor):
    if attribute.startswith("_"):
        continue
    print(attribute)

## Create table and data

In [ ]:
# Create a customer table.
cursor.execute("""
    CREATE TABLE Klanten (
        KlantId INTEGER PRIMARY KEY,
        Voornaam TEXT,
        Achternaam TEXT NOT NULL,
        Geboortedatum DATE
    );
""")

In [ ]:
# No rowcount for CREATE queries.
cursor.rowcount

In [ ]:
# Insert two new customers.
cursor.execute("""
    INSERT INTO Klanten
        (Voornaam, Achternaam, Geboortedatum)
    VALUES
        ('Ingrid', 'Jansen', '1984-09-06'),
        ('Henk', 'Knol', '1957-08-23')
    ;
""")

In [ ]:
# Verify the number of rows inserted...
# Only works for INSERT / UPDATE / REPLACE / DELETE.
cursor.rowcount

In [ ]:
# Get the last generated KlantId.
cursor.lastrowid

## Select data

In [ ]:
# Perform a SELECT query.
cursor.execute("SELECT * FROM Klanten")

In [ ]:
# The rowcount is not set...
cursor.rowcount

In [ ]:
# Use fetchone to get a single record.
record = cursor.fetchone()
record

In [ ]:
# Record is a simple tuple.
type(record)

In [ ]:
# Look at the cursor metadata:
# - name
# - type_code
# - display_size
# - internal_size
# - precision
# - scale
# - null_ok
cursor.description

In [ ]:
# Extract column names from the description.
column_names = [field_meta[0] for field_meta in cursor.description]
column_names

In [ ]:
# Convert record to dict.
dict(zip(column_names, record))

## Row Factory

In [18]:
# Set a row factory.
connection.row_factory = sqlite3.Row

In [19]:
# Create a new cursor using the row factory.
cursor = connection.cursor()

In [20]:
# Perform a query and fetch the first row.
cursor.execute("SELECT * FROM Klanten;")
row = cursor.fetchone()

In [ ]:
# Note that result is now a Row object.
type(row)

In [ ]:
# Get the column names from the Row object.
row.keys()

In [ ]:
# Index by column name.
row["Voornaam"]

In [ ]:
# Or by index.
row[1]